### Immersive Labs - Bursary end of course certificates

## Export Data

The data for the certificates is exported from https://modstudents.immersivelabs.online using the __Reports__ and the __Data Explorer__ In the __Lab Activity__ option it is possible to choose the fields requred and exprot as a CSV file.
_Make sure the date range matches the course duration as this defaults to the previous month only_
https://modstudent.immersivelabs.online/reports/data-explorer#lab-activity

These are the columns that were selected for the CSV file

	EMAIL	TEAMS	LAB_TITLE	CATEGORIES	ATTEMPT_COMPLETED_AT	LAB_ACCURACY	ACTIVE_MINUTES_IN_LAB	FULL_NAME	CPE_POINTS_EARNED	POINTS_FROM_LAB



In [36]:
import pandas as pd
from docx import Document
from docx.shared import Cm

# 1. Read in the CSV file and file containing info on part 1 complete
df_results = pd.read_csv("lab_activity.csv")
df_assignment = pd.read_csv("assigned_collections_completed.csv")

# Merge together by FULL_NAME
df_merged = df_results.merge(df_assignment[['FULL_NAME', 'IS_ASSIGNED_COLLECTION_COMPLETE']], 
                             on='FULL_NAME', how='left')


In [37]:

# 2. Group by the student's name. 
#    You could also include other columns in the group if you have multiple "FULL_NAME" collisions.
grouped = df_merged.groupby("FULL_NAME")

# Create a new Word document
doc = Document()

# Set margins in centimetres
for section in doc.sections:
    section.top_margin = Cm(2)      # 2 cm
    section.bottom_margin = Cm(1.5) # 1.5 cm
    section.left_margin = Cm(2)     # 2 cm
    section.right_margin = Cm(2)    # 2 cm

In [ ]:

# 3. Iterate over each student group
for name, group in grouped:
    # Extract the college
    college = group["TEAMS"].iloc[0]
    # College Name clean up string
    college = college.split('/',1)[1].strip()

    # Sum total points and total time - force points to an Int
    total_points = int(group["POINTS_FROM_LAB"].sum())
    total_time = group["ACTIVE_MINUTES_IN_LAB"].sum()

    # Format time from minutes into hours and minutes
    total_minutes = int(round(total_time))
    hours = total_minutes // 60
    minutes = total_minutes % 60
    if minutes == 1:
        minutes_string = f"{minutes} minute"
    else:
        minutes_string = f"{minutes} minutes"

    if hours > 0:
        time_string = f"{hours} hours {minutes_string}"
    else:
        time_string = minutes_string
    
    # Remove labs completed ore than once - that have no points awarded
    group = group[group["POINTS_FROM_LAB"] > 0]

    # Count how many labs this student completed
    num_labs = len(group)

    # 4. Add a heading for this student
    doc.add_heading(f"{name}", level=1)
    doc.add_paragraph(f"{college}")
    # doc.add_paragraph(f"Total Labs Completed: {num_labs}    |    Total Points: {total_points}    |    Total Time: {time_string}").bold = True
    stats = doc.add_paragraph("Total Labs Completed:")
    stats.add_run(f" {num_labs}").bold = True
    stats.add_run("    |    Total Points: ")
    stats.add_run(f"{total_points}").bold = True
    stats.add_run("    |    Total Time: ")
    stats.add_run(f"{time_string}").bold = True

    # If completed part 1 add
    is_complete = group["IS_ASSIGNED_COLLECTION_COMPLETE"].iloc[0]
    if is_complete:
        completion = doc.add_paragraph("Digital Skills 4 Defence: STEM and Digital Skills Bursary Cyber Fundamentals: ")
        completion.add_run("Successfully Completed").bold = True

    # 5. List out each lesson completed
    doc.add_heading("Labs Completed:", level=2)
    # Want to sort alphabetically first
    lab_completed = [(row["LAB_TITLE"], row["CATEGORIES"])
                    for _, row in group.iterrows()]

    lab_completed.sort(key=lambda x: x[0]) # Sort by lab title

    for lab_title, category in lab_completed:
            doc.add_paragraph(f"{lab_title} ({category})", style="List Bullet")

    # for idx, row in group.iterrows():
    #     lab_title = row["LAB_TITLE"]
    #     category   = row["CATEGORIES"]  # or row["LAB_ACCURACY"] if that represents difficulty
    #     doc.add_paragraph(
    #         f"{lab_title} ({category})", 
    #         style="List Bullet"
    #     )

    # Optional: Add a page break after each student's section
    doc.add_page_break()

# Finally, save the Word document
doc.save("lab_results_output.docx")
